In [ ]:
##
#Import Packages
import numpy as np
import sys
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
##
#Define Path to Code Database
DirPath = '/PATH/TO/bin/'

#Define Path to Example Data
DataPath = '/PATH/TO/ExampleData/'

#Define Output Path
OutputPath = '/PATH/TO/Output/'

In [ ]:
##
#Import Custom Functions
sys.path.append(DirPath)
from ImportData import *
from FreedAnalytical import *

In [ ]:
##
#Define Default Properties

##
#Define Diffusion Tensor (mm/s) (Dxx, Dyy, Dzz, Dxy, Dxz, Dyz) - Equivalent to Values in a Corpus Callosum Voxel
D = np.array([1.6254e-04,  8.0779e-05,  8.1765e-05, -4.0544e-05, -3.9950e-05, 2.1640e-05])

##
#Define Relaxation Times & B1 (Mean Values in Post-Mortem Brain)
T1 = 650
T2 = 35
B1 = 1

##
#Define Comparison Relaxation Times & B1
T1Comp = 800
T2Comp = 60
B1Comp = 0.5


In [ ]:
##
#Load Data
    # bvecs - bvectors (3xn)
    # FlipAngles - Flip Angles (degrees) (1xn)
    # tau - Diffusion Gradient Duration (seconds) (1xn), 
    # G - Diffusion Gradient Duration (G/cm - Equivalent to mT/m Divided by 10) (1xn)
    # TRs - Repetition Times (seconds) (1xn)
    # b0s - Array Defining b0 locations (b0 = 1, dwi = 0) (1xn)

bvecs, FlipAngles, tau, G, TRs, b0s = ImportTextDataDWSSFP(DataPath)

In [ ]:
##
#Select Volumes Corresponding to the First Flip Angle (24o) 
bvecs = bvecs[:,FlipAngles==FlipAngles[0]]
tau = tau[FlipAngles==FlipAngles[0]]
G = G[FlipAngles==FlipAngles[0]]
TRs = TRs[FlipAngles==FlipAngles[0]]
b0s = b0s[FlipAngles==FlipAngles[0]]
FlipAngles = FlipAngles[FlipAngles==FlipAngles[0]]

In [ ]:
##
#Forward Simulate Comparison Signal
S = FreedDWSSFPTensor(G, tau, TRs, FlipAngles*B1, T1, T2, bvecs, *D)

In [ ]:
##
#Estimate T1, T2 & B1 Leading to Identical Signal Profile
Dependencies, _ = curve_fit(lambda x, *theta: FreedDWSSFPTensor_curve_fit(x, theta, G, tau, TRs, FlipAngles, bvecs, B1Comp, T1Comp, T2Comp, Norm = True), 1, S/S[0], p0 = D/2 ,method='trf',maxfev=10**6,ftol=1e-12, xtol=1e-12, gtol=1e-12)

In [ ]:
##
#Reconstruct Signal
SRecon = FreedDWSSFPTensor(G, tau, TRs, FlipAngles*B1Comp, T1Comp, T2Comp, bvecs, *Dependencies)

##
#Estimate Ratios of Diffusion Coefficients
DRatio = D/Dependencies

In [ ]:
##
#Plot Degenerate Signals (Only Plot One b0)
fig = plt.figure()
idx = np.flatnonzero(b0s)[-1]
plt.plot(S[idx:]/S[idx],linewidth=2)
plt.plot(SRecon[idx:]/SRecon[idx],linestyle='--',linewidth=2)
plt.yticks([0,1])
plt.ylim([0,1])
plt.xticks([0,int(S[idx+1:].shape[0])/2,S[idx+1:].shape[0]], size=12)
plt.ylabel('Diffusion Attenuation (Normalised)', size=12)
plt.xlabel('Direction Number', size=12)
plt.legend([''.join([r'$\mathbf{D} = \{D_{xx}, D_{yy}, D_{zz}, D_{xy}, D_{xz}, D_{yz}$}']), 
            ''.join([r'$\mathbf{D} = \{$D$_{xx}$/',str(round(DRatio[0],1)),r', D$_{yy}$/',str(round(DRatio[1],1)),r', D$_{zz}$/',str(round(DRatio[2],1)),r', D$_{xy}$/',str(round(DRatio[3],1)),r', D$_{xy}$/',str(round(DRatio[4],1)),r', D$_{yz}$/',str(round(DRatio[5],1)),'}'])])

In [ ]:
#Save Figure
fig.savefig(''.join([OutputPath,'Figure2.pdf']),dpi=300,format='pdf',bbox_inches='tight')